## 从IGDB数据库爬取游戏种类和发行时间

In [ ]:
## 提取所有游戏的列表
import pandas as pd

file_path = "Twitch_game_data.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')
unique_games = df["Game"].unique()

unique_games_df = pd.DataFrame(unique_games, columns=["Game"])
unique_games_path = "unique_games.csv"
unique_games_df.to_csv(unique_games_path, index=False, encoding='ISO-8859-1')

print(f"Extracted {len(unique_games)} unique games. Saved to {unique_games_path}.")

Extracted 2360 unique games. Saved to unique_games.csv.


In [4]:
import requests
import pandas as pd
import time

# 你的 IGDB API 凭据
CLIENT_ID = "e1cvubjaunqp8g8rhyxmuog7ld4tsy"  # 这里替换成你的 Client ID
CLIENT_SECRET = "tnettt8ysiuybxxi1g8dlkzlwbzice"  # 这里替换成你的 Client Secret

# 获取 OAuth 令牌
def get_access_token():
    url = "https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials"
    }
    response = requests.post(url, params=params)
    response.raise_for_status()
    return response.json()["access_token"]

# 查询 IGDB API 获取游戏分类和上市日期
def get_game_info(access_token, game_name):
    url = "https://api.igdb.com/v4/games"
    headers = {
        "Client-ID": CLIENT_ID,
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }
    data = f'fields name, genres.name, first_release_date; where name ~ "{game_name}"*; limit 1;'
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()
    results = response.json()
    
    if results:
        game = results[0]
        genres = game.get("genres", [])
        genre_names = ", ".join([genre["name"] for genre in genres]) if genres else "Unknown"
        release_date = game.get("first_release_date")
        release_year_month = time.strftime('%Y-%m', time.gmtime(release_date)) if release_date else "Unknown"
        return genre_names, release_year_month
    return "Unknown", "Unknown"

# 读取唯一游戏列表
unique_games_path = "unique_games.csv"
df_games = pd.read_csv(unique_games_path, encoding='ISO-8859-1')

# 获取访问令牌
access_token = get_access_token()

# 创建游戏信息字典
game_info_dict = {}
for game in df_games["Game"].tolist():
    genre, release_date = get_game_info(access_token, game)
    game_info_dict[game] = {"Genre": genre, "Release Date": release_date}

# 生成包含游戏信息的 DataFrame
df_game_info = pd.DataFrame.from_dict(game_info_dict, orient='index').reset_index()
df_game_info.columns = ["Game", "Genre", "Release Date"]

# 保存更新后的游戏信息文件
output_path = "game_info.csv"
df_game_info.to_csv(output_path, index=False, encoding='ISO-8859-1')
print(f"Updated file saved: {output_path}")

Updated file saved: game_info.csv


In [ ]:
import requests
import pandas as pd
import time
import re
from tqdm import tqdm

# IGDB API 凭据
CLIENT_ID = "e1cvubjaunqp8g8rhyxmuog7ld4tsy"
CLIENT_SECRET = "tnettt8ysiuybxxi1g8dlkzlwbzice"

# 获取 OAuth 令牌
def get_access_token():
    url = "https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials"
    }
    response = requests.post(url, params=params)
    response.raise_for_status()
    return response.json()["access_token"]

# 查询 IGDB API 获取游戏信息
def get_game_info(access_token, game_name):
    url = "https://api.igdb.com/v4/games"
    headers = {
        "Client-ID": CLIENT_ID,
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }
    
    # 首先尝试精确匹配
    data = f'fields name, genres.name, first_release_date; where name = "{game_name}"; limit 5;'
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()
    results = response.json()
    
    # 如果没有精确匹配结果，使用搜索功能
    if not results:
        data = f'fields name, genres.name, first_release_date; search "{game_name}"; limit 5;'
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        results = response.json()
    
    if results:
        # 输出所有返回的结果，用于调试
        print(f"\n游戏名称: {game_name} 找到 {len(results)} 个结果:")
        for i, game in enumerate(results):
            release_date = game.get("first_release_date")
            release_str = time.strftime('%Y-%m', time.gmtime(release_date)) if release_date else "Unknown"
            print(f"  {i+1}. {game['name']} - 发布日期: {release_str}")
        
        # 保留第一个结果作为最佳匹配
        game = results[0]
        genres = game.get("genres", [])
        genre_names = ", ".join([genre["name"] for genre in genres]) if genres else "Unknown"
        release_date = game.get("first_release_date")
        
        if release_date:
            release_year = time.strftime('%Y', time.gmtime(release_date))
            release_month = time.strftime('%m', time.gmtime(release_date))
            return {
                "Genre": genre_names, 
                "Release Year": release_year,
                "Release Month": release_month,
                "Game Name in IGDB": game["name"]
            }
    
    return {
        "Genre": "Unknown", 
        "Release Year": "Unknown",
        "Release Month": "Unknown",
        "Game Name in IGDB": "Not Found"
    }

# 主函数：测试前50个游戏
def test_first_50_games():
    # 读取唯一游戏列表
    unique_games_path = "unique_games.csv"
    try:
        df_games = pd.read_csv(unique_games_path, encoding='UTF-8')
    except UnicodeDecodeError:
        df_games = pd.read_csv(unique_games_path, encoding='ISO-8859-1')
    
    # 获取前50个游戏
    test_games = df_games["Game"].head(50).tolist()
    
    # 获取访问令牌
    print("获取API访问令牌...")
    access_token = get_access_token()
    
    # 测试前50个游戏
    print("测试前50个游戏的信息获取...")
    test_results = []
    
    for game in tqdm(test_games, desc="测试游戏"):
        info = get_game_info(access_token, game)
        test_results.append({
            "Game": game,
            "Genre": info["Genre"],
            "Release Year": info["Release Year"],
            "Release Month": info["Release Month"],
            "Game Name in IGDB": info["Game Name in IGDB"]
        })
        time.sleep(0.3)  # 防止API限制
        
    df_test = pd.DataFrame(test_results)
    df_test.to_csv("test_first_50_games.csv", index=False, encoding='UTF-8')
    print("测试完成，结果已保存至 test_first_50_games.csv")
    
    # 显示测试结果摘要
    print("\n测试结果摘要:")
    print(df_test[["Game", "Release Year", "Release Month", "Game Name in IGDB"]].head(10))
    
    unknown_count = sum(df_test["Release Year"] == "Unknown")
    print(f"\n总共测试了 {len(df_test)} 个游戏，其中 {unknown_count} 个没有找到发行日期 ({unknown_count/len(df_test)*100:.1f}%)")

if __name__ == "__main__":
    test_first_50_games()

获取API访问令牌...
测试前50个游戏的信息获取...


测试游戏:   0%|          | 0/50 [00:00<?, ?it/s]


游戏名称: League of Legends 找到 1 个结果:
  1. League of Legends - 发布日期: 2009-10


测试游戏:   2%|▏         | 1/50 [00:00<00:32,  1.49it/s]


游戏名称: Counter-Strike: Global Offensive 找到 1 个结果:
  1. Counter-Strike: Global Offensive - 发布日期: 2012-08


...


游戏名称: Euro Truck Simulator 2 找到 1 个结果:
  1. Euro Truck Simulator 2 - 发布日期: 2012-10


测试游戏:  98%|█████████▊| 49/50 [00:29<00:00,  1.80it/s]


游戏名称: The Legend of Zelda: Majora's Mask 找到 1 个结果:
  1. The Legend of Zelda: Majora's Mask - 发布日期: 2000-04


测试游戏: 100%|██████████| 50/50 [00:30<00:00,  1.66it/s]

测试完成，结果已保存至 test_first_50_games.csv

测试结果摘要:
                               Game Release Year Release Month  \
0                 League of Legends         2009            10   
1  Counter-Strike: Global Offensive         2012            08   
2                            Dota 2         2013            07   
3                       Hearthstone         2014            03   
4       Call of Duty: Black Ops III         2015            11   
5                         Minecraft         2016            12   
6                 World of Warcraft         2004            11   
7                 Z1: Battle Royale         2018            02   
8             Talk Shows & Podcasts      Unknown       Unknown   
9                           FIFA 16         2015            09   

                  Game Name in IGDB  
0                 League of Legends  
1  Counter-Strike: Global Offensive  
2                            Dota 2  
3                       Hearthstone  
4       Call of Duty: Black Ops III  


In [ ]:
import requests
import pandas as pd
import time
import re
from tqdm import tqdm

# IGDB API 凭据
CLIENT_ID = "e1cvubjaunqp8g8rhyxmuog7ld4tsy"
CLIENT_SECRET = "tnettt8ysiuybxxi1g8dlkzlwbzice"

# 获取 OAuth 令牌
def get_access_token():
    url = "https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials"
    }
    response = requests.post(url, params=params)
    response.raise_for_status()
    return response.json()["access_token"]

# 查询 IGDB API 获取游戏信息，使用更精确的查询，并返回多个搜索结果数量
def get_game_info(access_token, game_name):
    url = "https://api.igdb.com/v4/games"
    headers = {
        "Client-ID": CLIENT_ID,
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }
    
    # 使用更精确的查询条件，并请求更多相关信息
    # 首先尝试完全匹配
    data = f'fields name, genres.name, first_release_date, rating, status, platforms.name; where name = "{game_name}"; limit 5;'
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()
    exact_results = response.json()
    
    # 如果没有完全匹配的结果，尝试模糊匹配
    search_results = []
    if not exact_results:
        # 清理游戏名称，删除特殊字符
        clean_name = re.sub(r'[^\w\s]', '', game_name)
        data = f'fields name, genres.name, first_release_date, rating, status, platforms.name; search "{clean_name}"; limit 5;'
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        search_results = response.json()
    
    # 合并结果 - 如果有精确匹配，就用精确匹配；否则用搜索结果
    results = exact_results if exact_results else search_results
    results_count = len(results)
    
    if results:
        # 选择评分最高或发布最早的游戏
        best_match = None
        highest_rating = -1
        
        for game in results:
            # 确保名称相似度高
            if game["name"].lower() == game_name.lower() or game["name"].lower() in game_name.lower() or game_name.lower() in game["name"].lower():
                rating = game.get("rating", 0)
                if rating > highest_rating:
                    highest_rating = rating
                    best_match = game
        
        # 如果没有找到好的匹配，选择第一个结果
        if not best_match and results:
            best_match = results[0]
            
        if best_match:
            genres = best_match.get("genres", [])
            genre_names = ", ".join([genre["name"] for genre in genres]) if genres else "Unknown"
            release_date = best_match.get("first_release_date")
            
            if release_date:
                # 将Unix时间戳转换为年份和月份
                release_year = time.strftime('%Y', time.gmtime(release_date))
                release_month = time.strftime('%m', time.gmtime(release_date))
                return {
                    "Genre": genre_names, 
                    "Release Year": release_year,
                    "Release Month": release_month,
                    "Game Name in IGDB": best_match["name"],
                    "Multiple Results": "是" if results_count > 1 else "否",
                    "Results Count": results_count
                }
    
    return {
        "Genre": "Unknown", 
        "Release Year": "Unknown",
        "Release Month": "Unknown",
        "Game Name in IGDB": "Not Found",
        "Multiple Results": "否",
        "Results Count": 0
    }

# 主函数：处理所有游戏
def process_all_games(df_games, access_token):
    print(f"开始处理全部 {len(df_games)} 个游戏...")
    game_info_list = []
    
    for game in tqdm(df_games["Game"].tolist(), desc="获取游戏信息"):
        try:
            info = get_game_info(access_token, game)
            game_info_list.append({
                "Game": game,
                "Genre": info["Genre"],
                "Release Year": info["Release Year"],
                "Release Month": info["Release Month"],
                "Game Name in IGDB": info["Game Name in IGDB"],
                "Multiple Results": info["Multiple Results"],
                "Results Count": info["Results Count"]
            })
            # 添加随机延迟0.2-0.5秒，防止API限制
            time.sleep(0.2 + 0.3 * (time.time() % 1))
        except Exception as e:
            print(f"\n处理游戏 '{game}' 时出错: {str(e)}")
            # 如果出错，添加空结果并继续
            game_info_list.append({
                "Game": game,
                "Genre": "Error",
                "Release Year": "Error",
                "Release Month": "Error",
                "Game Name in IGDB": f"Error: {str(e)}",
                "Multiple Results": "否",
                "Results Count": 0
            })
            # 发生错误时稍微多等一下
            time.sleep(1)
        
        # 每处理100个游戏保存一次中间结果，防止意外中断导致数据丢失
        if len(game_info_list) % 100 == 0:
            temp_df = pd.DataFrame(game_info_list)
            temp_df.to_csv(f"game_info_temp_{len(game_info_list)}.csv", index=False, encoding='UTF-8')
            print(f"\n已处理 {len(game_info_list)} 个游戏，临时保存结果")
            
    df_game_info = pd.DataFrame(game_info_list)
    output_path = "game_info_full.csv"
    df_game_info.to_csv(output_path, index=False, encoding='UTF-8')
    print(f"处理完成，文件已保存至 {output_path}")
    
    # 输出统计信息
    multiple_results = df_game_info[df_game_info["Multiple Results"] == "是"].shape[0]
    not_found = df_game_info[df_game_info["Game Name in IGDB"] == "Not Found"].shape[0]
    errors = df_game_info[df_game_info["Release Year"] == "Error"].shape[0]
    
    print("\n处理统计:")
    print(f"总游戏数: {len(df_game_info)}")
    print(f"有多个搜索结果的游戏: {multiple_results} ({multiple_results/len(df_game_info)*100:.1f}%)")
    print(f"未找到的游戏: {not_found} ({not_found/len(df_game_info)*100:.1f}%)")
    print(f"处理出错的游戏: {errors} ({errors/len(df_game_info)*100:.1f}%)")
    
    return df_game_info

if __name__ == "__main__":
    # 读取唯一游戏列表
    unique_games_path = "unique_games.csv"
    try:
        df_games = pd.read_csv(unique_games_path, encoding='UTF-8')
    except UnicodeDecodeError:
        df_games = pd.read_csv(unique_games_path, encoding='ISO-8859-1')
    
    # 获取访问令牌
    print("获取API访问令牌...")
    access_token = get_access_token()
    
    # 询问用户是否想处理整个文件或部分文件
    while True:
        process_option = input("请选择处理方式:\n1. 处理全部游戏\n2. 处理指定数量的游戏\n请输入选择 (1/2): ")
        if process_option == '1':
            # 处理所有游戏
            df_game_info = process_all_games(df_games, access_token)
            break
        elif process_option == '2':
            try:
                num_games = int(input("请输入要处理的游戏数量: "))
                if num_games > 0 and num_games <= len(df_games):
                    # 处理指定数量的游戏
                    df_partial = df_games.head(num_games)
                    df_game_info = process_all_games(df_partial, access_token)
                    break
                else:
                    print(f"请输入有效数字 (1-{len(df_games)})")
            except ValueError:
                print("请输入有效数字")
        else:
            print("无效选择，请重新输入")
    
    print("处理完成！")

获取API访问令牌...
开始处理全部 2360 个游戏...


获取游戏信息:   4%|▍         | 100/2360 [01:10<27:25,  1.37it/s]


已处理 100 个游戏，临时保存结果


获取游戏信息:   8%|▊         | 200/2360 [02:19<23:24,  1.54it/s]


已处理 200 个游戏，临时保存结果


...


已处理 2200 个游戏，临时保存结果


获取游戏信息:  97%|█████████▋| 2300/2360 [26:56<00:37,  1.62it/s]


已处理 2300 个游戏，临时保存结果


获取游戏信息: 100%|██████████| 2360/2360 [27:38<00:00,  1.42it/s]

处理完成，文件已保存至 game_info_full.csv

处理统计:
总游戏数: 2360
有多个搜索结果的游戏: 397 (16.8%)
未找到的游戏: 144 (6.1%)
处理出错的游戏: 1 (0.0%)
处理完成！


In [1]:
import requests
import pandas as pd
import time
import re
from tqdm import tqdm

# IGDB API 凭据
CLIENT_ID = "e1cvubjaunqp8g8rhyxmuog7ld4tsy"
CLIENT_SECRET = "tnettt8ysiuybxxi1g8dlkzlwbzice"

# 获取 OAuth 令牌
def get_access_token():
    url = "https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials"
    }
    response = requests.post(url, params=params)
    response.raise_for_status()
    return response.json()["access_token"]

# 查询 IGDB API 获取游戏信息
def get_game_info(access_token, game_name):
    url = "https://api.igdb.com/v4/games"
    headers = {
        "Client-ID": CLIENT_ID,
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }
    
    data = f'fields name, genres.name, first_release_date, rating, status, platforms.name; where name = "{game_name}"; limit 5;'
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()
    exact_results = response.json()
    
    search_results = []
    if not exact_results:
        clean_name = re.sub(r'[^\w\s]', '', game_name)
        data = f'fields name, genres.name, first_release_date, rating, status, platforms.name; search "{clean_name}"; limit 5;'
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        search_results = response.json()
    
    results = exact_results if exact_results else search_results
    results_count = len(results)
    
    if results:
        best_match = None
        highest_rating = -1
        
        for game in results:
            if game["name"].lower() == game_name.lower() or game["name"].lower() in game_name.lower() or game_name.lower() in game["name"].lower():
                rating = game.get("rating", 0)
                if rating > highest_rating:
                    highest_rating = rating
                    best_match = game
        
        if not best_match and results:
            best_match = results[0]
            
        if best_match:
            genres = best_match.get("genres", [])
            genre_names = ", ".join([genre["name"] for genre in genres]) if genres else "Unknown"
            release_date = best_match.get("first_release_date")
            
            if release_date:
                release_year = time.strftime('%Y', time.gmtime(release_date))
                release_month = time.strftime('%m', time.gmtime(release_date))
                return {
                    "Genre": genre_names, 
                    "Release Year": release_year,
                    "Release Month": release_month,
                    "Game Name in IGDB": best_match["name"],
                    "Multiple Results": "是" if results_count > 1 else "否",
                    "Results Count": results_count
                }
    
    return {
        "Genre": "Unknown", 
        "Release Year": "Unknown",
        "Release Month": "Unknown",
        "Game Name in IGDB": "Not Found",
        "Multiple Results": "否",
        "Results Count": 0
    }

# 处理所有游戏
def process_all_games(df_games, access_token):
    print(f"开始处理全部 {len(df_games)} 个游戏...")
    game_info_list = []
    
    for game in tqdm(df_games["Game"].tolist(), desc="获取游戏信息"):
        try:
            info = get_game_info(access_token, game)
            game_info_list.append({
                "Game": game,
                "Genre": info["Genre"],
                "Release Year": info["Release Year"],
                "Release Month": info["Release Month"],
                "Game Name in IGDB": info["Game Name in IGDB"],
                "Multiple Results": info["Multiple Results"],
                "Results Count": info["Results Count"]
            })
            time.sleep(0.2 + 0.3 * (time.time() % 1))
        except Exception as e:
            print(f"\n处理游戏 '{game}' 时出错: {str(e)}")
            game_info_list.append({
                "Game": game,
                "Genre": "Error",
                "Release Year": "Error",
                "Release Month": "Error",
                "Game Name in IGDB": f"Error: {str(e)}",
                "Multiple Results": "否",
                "Results Count": 0
            })
            time.sleep(1)
            
    df_game_info = pd.DataFrame(game_info_list)
    output_path = "igdb_data.csv"
    df_game_info.to_csv(output_path, index=False, encoding='UTF-8')
    print(f"处理完成，文件已保存至 {output_path}")
    
    return df_game_info

if __name__ == "__main__":
    unique_games_path = "unique_games.csv"
    try:
        df_games = pd.read_csv(unique_games_path, encoding='UTF-8')
    except UnicodeDecodeError:
        df_games = pd.read_csv(unique_games_path, encoding='ISO-8859-1')
    
    print("获取API访问令牌...")
    access_token = get_access_token()
    
    while True:
        process_option = input("请选择处理方式:\n1. 处理全部游戏\n2. 处理指定数量的游戏\n请输入选择 (1/2): ")
        if process_option == '1':
            df_game_info = process_all_games(df_games, access_token)
            break
        elif process_option == '2':
            try:
                num_games = int(input("请输入要处理的游戏数量: "))
                if num_games > 0 and num_games <= len(df_games):
                    df_partial = df_games.head(num_games)
                    df_game_info = process_all_games(df_partial, access_token)
                    break
                else:
                    print(f"请输入有效数字 (1-{len(df_games)})")
            except ValueError:
                print("请输入有效数字")
        else:
            print("无效选择，请重新输入")
    
    print("处理完成！")

获取API访问令牌...
开始处理全部 2360 个游戏...


获取游戏信息:  52%|█████▏    | 1218/2360 [14:25<16:05,  1.18it/s]


处理游戏 'nan' 时出错: expected string or bytes-like object, got 'float'


获取游戏信息: 100%|██████████| 2360/2360 [27:35<00:00,  1.43it/s]

处理完成，文件已保存至 igdb_data.csv
处理完成！
